**Klasifikavimas naudojant spektrogramas**

##Reikalingi import'ai

In [1]:
!pip install git+https://github.com/huggingface/datasets -q
!pip install librosa

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
import torchaudio
from torchvision import transforms
import matplotlib.pyplot as plt
from IPython.display import Audio
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import librosa
import librosa.core
import librosa.core.spectrum as spec
import torchvision
import os
import random
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [3]:
print(torch.cuda.is_available())
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

True
cuda:0


In [4]:
ds = load_dataset("MLCommons/ml_spoken_words", languages=["lv"])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset ml_spoken_words downloaded and prepared to /root/.cache/huggingface/datasets/MLCommons___ml_spoken_words/lv_wav-28bc8550fcb4f8cb/1.0.0/321ea853cf0a05abb7a2d7efea900692a3d8622af65a2f3ce98adb7800a5d57b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

##Duomenų pasiruošimas

In [8]:
def convertToImage(file, gender, state, is_train=False):
    if gender == 0:
        gender_path = "male"
    elif gender == 1:
        gender_path = "female"
    else:
        gender_path = "other"

    if not os.path.exists(f"./{state}/{gender_path}"):
        os.makedirs(f"./{state}/{gender_path}")

    y, sr = librosa.load(file)
    if is_train:
        if random.random() > 0.5:
            noise_path = "/content/noise.wav"
            noise, _ = librosa.load(noise_path)
            noise = noise[:y.shape[0]]
            y = y + noise

    window_size = 1024
    window = np.hanning(window_size)
    stft = spec.stft(y, n_fft=window_size, hop_length=512, window=window)
    out = 2 * np.abs(stft) / np.sum(window)

    fig = plt.Figure()
    canvas = FigureCanvas(fig)
    ax = fig.add_subplot(111)
    p = librosa.display.specshow(
        librosa.amplitude_to_db(out, ref=np.max), ax=ax, y_axis="log", x_axis="time"
    )
    os.makedirs(f"./{state}/{gender_path}", exist_ok=True)
    fig.savefig(
        f"./{state}/{gender_path}/{os.path.splitext(os.path.basename(file))[0]}.png"
    )

In [9]:
def prepareData(subset, state, is_train = False):
  if not os.path.exists(state):
      os.makedirs(state)
  for file in subset:
    if file["gender"] != 3:
      convertToImage(file["file"], file["gender"], state, is_train)

In [10]:
prepareData(ds["train"], "train", is_train=True)
prepareData(ds["validation"], "valid", is_train=False)
prepareData(ds["test"], "test", is_train=False)

In [11]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.classes = os.listdir(data_dir)
        self.samples = []
        self.class_to_idx = []

        for label, class_name in enumerate(self.classes):
            self.class_to_idx.append([class_name, label])
            cls_dir = os.path.join(data_dir, class_name)
            for file_name in os.listdir(cls_dir):
                file_path = os.path.join(cls_dir, file_name)
                self.samples.append((file_path, label))

    def transform(self, img):
        img = transforms.functional.resize(img, size=(201,81))
        img = transforms.functional.to_tensor(img)
        return img

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        with open(img_path, 'rb') as f:
            img = Image.open(f).convert('RGB')
        img = self.transform(img)
        return img, label

In [12]:
train_data = CustomDataset("train")
valid_data = CustomDataset("valid")
test_data = CustomDataset("test")

In [13]:
num_workers = 2
batch_size = 16

train_loader = DataLoader(train_data, batch_size = batch_size, num_workers = num_workers, shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = batch_size, num_workers = num_workers, shuffle = True)
test_loader = DataLoader(test_data, batch_size = batch_size, num_workers = num_workers, shuffle = False)

print(f'Train: {len(train_data)}, Validation: {len(valid_data)}, Test: {len(test_data)}')

Train: 9531, Validation: 1314, Test: 1266


##Modelis


In [14]:
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(51136, 50)
        self.fc2 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1) 

In [16]:
model = CNNet().to(device)

##Treniravimas


In [23]:
def train(dataloader, model, loss, optimizer):
    model.train()
    loss_acum = np.array([], dtype = np.float32)
    for img, label in dataloader:
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        pred = model(img)
        loss = cost(pred, label)
        loss_acum = np.append(loss_acum, loss.cpu().detach().numpy())
        loss.backward()
        optimizer.step()

    print(f'Training loss: {np.mean(loss_acum)}')

def evaluate(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    loss_acum = np.array([], dtype = np.float32)

    with torch.no_grad():
        for batch, (img, label) in enumerate(dataloader):
            img, label = img.to(device), label.to(device)
            pred = model(img)

            test_loss = cost(pred, label)
            loss_acum = np.append(loss_acum, test_loss.cpu().detach().numpy())
            correct += (pred.argmax(1)==label).type(torch.float).sum().item()

    loss = np.mean(loss_acum)
    correct /= size

    print(f'\nValidation:\nacc: {(100*correct):>0.1f}%, loss: {loss}\n')

In [18]:
epochs = 10
cost = torch.nn.CrossEntropyLoss()

learning_rate = 0.00005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train(train_loader, model, cost, optimizer)
    evaluate(valid_loader, model)
print('Done!')

Epoch 1
-------------------------------
Training loss: 0.4352494180202484

Validation:
acc: 68.0%, loss: 0.5520985722541809

Epoch 2
-------------------------------
Training loss: 0.3435889184474945

Validation:
acc: 75.8%, loss: 0.3817436695098877

Epoch 3
-------------------------------
Training loss: 0.2781955301761627

Validation:
acc: 77.2%, loss: 0.36550578474998474

Epoch 4
-------------------------------
Training loss: 0.24140848219394684

Validation:
acc: 86.1%, loss: 0.3332992196083069

Epoch 5
-------------------------------
Training loss: 0.2099907398223877

Validation:
acc: 90.3%, loss: 0.26451241970062256

Epoch 6
-------------------------------
Training loss: 0.18901817500591278

Validation:
acc: 90.6%, loss: 0.283707857131958

Epoch 7
-------------------------------
Training loss: 0.18123795092105865

Validation:
acc: 92.8%, loss: 0.22160984575748444

Epoch 8
-------------------------------
Training loss: 0.16540499031543732

Validation:
acc: 92.5%, loss: 0.230823695659

##Testavimas


Metrikos

In [19]:
class_num = len(train_loader.dataset.classes)
classes_names, class_label = zip(*train_loader.dataset.class_to_idx)

conf_matrix = torch.zeros(class_num, class_num)
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.to(device))
        _, predicted = torch.max(outputs.data, 1)

        # Increment the corresponding entry in the confusion matrix
        for i in range(len(predicted)):
            conf_matrix[labels[i], predicted[i]] += 1

conf_matrix = conf_matrix.numpy()

accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)

precision = np.zeros(class_num)
recall = np.zeros(class_num)
f1 = np.zeros(class_num)
for i in class_label:
    tp = conf_matrix[i, i]
    tn = np.sum(conf_matrix) - np.sum(conf_matrix[i, :]) - np.sum(conf_matrix[:, i]) + conf_matrix[i, i]
    fp = np.sum(conf_matrix[:, i]) - tp
    fn = np.sum(conf_matrix[i, :]) - tp
    precision[i] = tp / (tp + fp)
    recall[i] = tp / (tp + fn)
    f1[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])

micro_precision = np.mean(precision)
micro_recall = np.mean(recall)
micro_f1 = np.mean(f1)

print("Confusion matrix:")
print(conf_matrix)
print("Subset: Accuracy: {:.3f} , Precision: {:.3f}, Recall: {:.3f}, F1: {:.3f}".format(accuracy, micro_precision, micro_recall, micro_f1))
for i, class_name in enumerate(classes_names):
    print("Class {}: Precision: {:.3f}, Recall: {:.3f}, F1: {:.3f}".format(class_name, precision[i], recall[i], f1[i]))

Confusion matrix:
[[373.  52.]
 [ 42. 799.]]
Subset: Accuracy: 0.926 , Precision: 0.919, Recall: 0.914, F1: 0.916
Class female: Precision: 0.899, Recall: 0.878, F1: 0.888
Class male: Precision: 0.939, Recall: 0.950, F1: 0.944


Vieno garo įrašo konvertavimas į spektrogramą

In [21]:
audio_path = "voice.wav"
spectrogram_path = "spectrogram.png"

waveform, _ = librosa.load(audio_path)

window_size = 1024
window = np.hanning(window_size)
stft = spec.stft(waveform, n_fft=window_size, hop_length=512, window=window)
out = 2 * np.abs(stft) / np.sum(window)

fig = plt.Figure()
canvas = FigureCanvas(fig)
ax = fig.add_subplot(111)
p = librosa.display.specshow(
    librosa.amplitude_to_db(out, ref=np.max), ax=ax, y_axis="log", x_axis="time"
)

fig.savefig(spectrogram_path)

Testavimas su viena spectrograma

In [22]:
spectrogram_path = "spectrogram.png"
with open(spectrogram_path, 'rb') as f:
            img = Image.open(f).convert('RGB')
img = transforms.functional.resize(img, size=(201,81))
img = transforms.functional.to_tensor(img)

classes = test_data.class_to_idx

spectrogram = img.unsqueeze(0).to(device)

# Evaluate the model on the test data
with torch.no_grad():
    outputs = model(spectrogram)
    _, predicted = torch.max(outputs.data, 1)
    predicted_class = class_name = [class_name for class_name, index in classes if index == predicted.item()][0]
    print(f'Predicted class name: {predicted_class}')

Predicted class name: male
